In [35]:
import mysql.connector
import json
import gzip
#-*-coding:utf8-*-
#print('Importado')

In [36]:
class ConnectionConfig:
    
    def __init__(self, server, port,user, password, database):
        self.server = server
        self.port = port
        self.user = user
        self.password = password
        self.database = database

In [37]:
class MySQLEngine:
    def __init__(self,config):
        self.server = config.server
        self.port = config.port
        self.user = config.user
        self.password = config.password
        self.database = config.database
        

        self.start()
    def start(self):
        self.con = mysql.connector.connect(
            host = self.server,
            port = self.port,
            user = self.user,
            password = self.password,
            database = self.database
        )

        #Version del texto
        self.link = self.con.cursor()
    def select(self, query):
        self.link.execute(query)
        return self.link.fetchall()
    def management(self,sp, arg):
        temp = self.link.callproc(sp, arg)
        self.commit()
        return temp
        
    def commit(self):
        self.con.commit()

In [38]:
config = ConnectionConfig("localhost","3306","admin","admin","Respaldo") 
engie = MySQLEngine(config)
paint = engie.select("""SELECT 
                            var_name,
                           jso_data 
                        FROM
                            Paint
                        WHERE 
                            id_user = 1 AND CAST(AES_DECRYPT(var_name ,'admin')AS CHAR) = 'DIBUJOx'
                        ;""")
print(paint)

[]


In [39]:
"""for tupla in paint:
    print(tupla)"""

'for tupla in paint:\n    print(tupla)'

## Clase User
### Agregar usuario
Para agregar un nuevo usuario a la base de datos B se llamara al sp_addUser que ya se encuentra incorporado en la base de datos 

In [40]:
class User():
    def __init__(self):
        pass
    # Agregar usuario
    def addUser(self, userName, password, typex, penColor, fillColor):
        query = engie.management('sp_addUser',(userName,password,typex, penColor, fillColor,'admin', None))
    
              

In [41]:
User().addUser('Rosa','122345',1, '#12356','#123123')

## Clase Paint
### Agregar un dibujo


In [42]:
class paint():
    def __init__(self, idUser):
        self.idUser = idUser
        self.paints = [] #Digujos del usuario
        
    def search(self,namePaint): #Solicitar los dibujos que pertencen al usuario
        self.paints = engie.select("""SELECT 
                                        CAST(AES_DECRYPT(var_name,'admin')AS CHAR) ,
                                        CAST(AES_DECRYPT(jso_data,'admin')AS CHAR) 
                                    FROM
                                        Paint
                                    WHERE 
                                        id_user = %s AND CAST(AES_DECRYPT(var_name ,'admin')AS CHAR) = '%s'
                                    ;""" % (self.idUser,namePaint))
        return self.paints
                                   
    def addPaint(self, namePaint, data, idUser, adminPass= 'admin'):
        
        compress_data = self.compress_data(data)
        temp = compress_data.hex()
        query = engie.management('sp_addPaint',(namePaint,temp,idUser,'admin', None))

        if query[4] > 0:
            print('Dibujo ingresado con exito')
            return True
        else:
            print('ERROR al guardar')
            return False
        
    def compress_data(self, data):
        json_data = json.dumps(data,indent=1)
        enconde = json_data.encode('utf-8')
        compressd = gzip.compress(enconde)
        return compressd
    
    def decompress_data(self, data):
        return gzip.decompress(data)
    
    def showPaints(self):
        print(self.paints)
        return self.paints
    



In [43]:
pait = paint(1)
pait.addPaint('Pruebas123',{"VALOR":"JACOME","VALOR2":"rosalia", "VALOR3":"ABNERSON"},1,'admin')

Dibujo ingresado con exito


True

In [20]:
result = pait.search('Pruebas123')
temp = pait.decompress_data(bytes.fromhex(result[0][1]))
print(temp.decode())

{
 "VALOR": "JACOME",
 "VALOR2": "rosalia",
 "VALOR3": "ABNERSON"
}


In [17]:
a = gzip.compress('{"VALOR":"JACOME","VALOR2":"rosalia", "VALOR3":"ABNERSON"}'.encode())
b = str(a)
print('{"VALOR":"JACOME","VALOR2":"rosalia", "VALOR3":"ABNERSON"}'.encode())
gzip.decompress(B.replace("sss", ""))

b'{"VALOR":"JACOME","VALOR2":"rosalia", "VALOR3":"ABNERSON"}'


NameError: name 'B' is not defined